In [1]:
import ee
import geemap as emap

In [2]:
# Initialize and authenticate
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWhnf0TLIYDSXbURPocUbbPFYOq2-ZNNVICb2wY3re8n6NMFYccvRiY

Successfully saved authorization token.


In [ ]:
Map=emap.Map()
Map

# <font color="blue"> 1. Extract raster values from a single image (single time) <end>
    
The following code demonstrated how to extract raster values at a `given point` shapefile using various functions.

- **Extract rainfall using `.sampleRegions`**

In [ ]:
# Extract total rainfall data for the period from 1/1/2018 to 30/1/2018
thang1=ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterDate("2018-01-01","2018-01-30").sum()

In [ ]:
# locations needed to get rainfall data
point=ee.FeatureCollection("users/miketu72/Test_point")
# Center the point shafile 
Map.centerObject(point,12)
# Visualizing the point data
Map.addLayer(point,{},"Location")

Map

In [ ]:
# Extract total January rainfall using sample.Regions method
extractedData=thang1.sampleRegions(collection=point, scale=1000,geometries =True).getInfo()

In [ ]:
# Write a function to format extracted Data into a clean and tidy format dataframe
import pandas as pd
def dataFormat(inputDate):
    mlist=[]
    coord=[]
    for item in inputDate["features"]:
        mlist.append(item["properties"])
        coord.append(item["geometry"]["coordinates"])
    test=pd.DataFrame(mlist)
    test["coordiates"]=coord
    return test

In [ ]:
df=dataFormat(extractedData)
df.head() # There are some repeated same values in precipitation, this is due to a coarse resolution and two or more points fall in the same pixel

- **Extract mean temperature for a single image using `.sampleRegions`**

The below dataset provides hourly global temperature as well as dozens of other variables. If you are more interested in this dataset, please see this [link](https://cds.climate.copernicus.eu/cdsapp#!/dataset/10.24381/cds.e2161bac?tab=overview).

The task below aims to extract mean of temperature in January 2017 from hourly temperature dataset.

In [ ]:
# Extract mean temperature for the month of January 2017
tem=ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filterDate("2017-01-01","2017-01-30").select("temperature_2m").mean()

In [ ]:
extractedData=tem.sampleRegions(collection=point, scale=1000,geometries =True).getInfo()

In [ ]:
import pandas as pd
def dataFormat(inputDate):
    mlist=[]
    coord=[]
    for item in inputDate["features"]:
        mlist.append(item["properties"])
        coord.append(item["geometry"]["coordinates"])
    test=pd.DataFrame(mlist)
    test["coordiates"]=coord
    test["temperature_2m"]=test.temperature_2m-273.15
    return test

In [ ]:
# This dataset contains all variables in the dataset such as temperature at 2m. Some values had the same because those points fall 
# in the same pixel
df=dataFormat(extractedData)
df.head()

- **Extract Landsat surface reflectance `sampleRegions`**

In [ ]:
# Select Landsat 8 SR 
ls=ee.Image("LANDSAT/LC08/C01/T2_SR/LC08_127045_20170417")

In [ ]:
extractedLS=ls.sampleRegions(collection=point, scale=30,geometries =True).getInfo()

In [ ]:
import pandas as pd
def dataFormat(inputDate):
    mlist=[]
    coord=[]
    for item in inputDate["features"]:
        mlist.append(item["properties"])
        coord.append(tuple(item["geometry"]["coordinates"]))
    test=pd.DataFrame(mlist)
    test["coordiates"]=coord
    return test

In [ ]:
df=dataFormat(extractedLS)

df.head()

# <font color="blue"> Extract time series raster values <end>

- **Extract daily rainfal data using `.getRegion` function**

In [ ]:
# Define time period from 1/1/2018 - 30/1/2018
daily=ee.ImageCollection("ECMWF/ERA5/MONTHLY").filterDate("2018-01-01","2020-12-31")

In [ ]:
data=daily.getRegion(geometry=point, scale=1000).getInfo()

In [ ]:
df=pd.DataFrame(data[1:], columns=data[0])
df

In [ ]:
import datetime as dt
# This function aims to convert date code into human readable datetime. 
def timeConverter(date_code):
    start_date=dt.datetime(1970,1,1,0,0,0)
    hour_number=date_code/(60000*60)
    delta=dt.timedelta(hours=hour_number)
    end_date=start_date+delta
    return end_date
# This function to create a formated dataframe
import pandas as pd
def dataFormat(data):
    df=pd.DataFrame(data[1:], columns=data[0])
    df["time"]=[timeConverter(i) for i in df.time]
    return df
test=dataFormat(data)
test.head()

# <font color="blue"> Alternative method of extracting raster values for a single image </font>

The following demonstration focused on using `polygon` shapefile to extract raster values. The extracted value is a mean, median, min, max or other defined numbers of a given polygon.

In [3]:
# Load Vietnam map that consists of 63 provinces
VN=ee.FeatureCollection("users/miketu72/VN_Map")

**Daily rainfall and temperature data **

In [10]:
rainfall=ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterDate("2018-01-01","2019-01-01").select("precipitation").sum()

In [11]:
# MODIS temperature and convert it to degree
tem=ee.ImageCollection("MODIS/006/MOD11A2").filterDate("2018-01-01","2019-01-01").select("LST_Day_1km").mean().multiply(0.02).subtract(273.15)

In [12]:
# Combine two images into a 2-band image
stack=ee.Image([rainfall,tem]) # or ee.Image.cat([list of images]) 

- **Extract raster values for a given image using `.reduceRegions`**

In [14]:
value=stack.reduceRegions(collection=VN,reducer="mean",scale=1000) # Extract temperature and rainfall value for each province at a scale of 1km

In [16]:
# Select properties and remove geometry
mdict=value.select(["VARNAME_1","precipitation","LST_Day_1km"], retainGeometry=False).getInfo()

In [17]:
# Write a function to form a tidy dataframe
import pandas as pd
def dataFormat(input_data):
    data=mdict["features"]
    slist=[]
    for i in data:
        thuoctinh=i["properties"]
        slist.append(thuoctinh)
    df=pd.DataFrame(slist)
    return df
df=dataFormat(mdict)
df

,LST_Day_1km,VARNAME_1,precipitation
0,28.595952,An Giang,1266.575121
1,28.562227,Bac Lieu,1921.627244
2,27.434645,Bac Giang,1900.901591
3,24.869136,Bac Kan,1961.718338
4,28.490077,Bac Ninh,1828.250281
...,...,...,...
58,29.204907,Can Tho,1337.133902
59,28.398911,Da Nang,2061.819433
60,27.902961,Hai Phong,2129.469908
61,31.815883,Ho Chi Minh,1808.097697


- **Map over the collection using `reduceRegions()` for a given period of time**

In [20]:
def years(year):
    start=ee.Date.fromYMD(year,1,1)
    end=start.advance(1, "year")
    rainfall=ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterDate(start,end).select("precipitation").sum()
    value=rainfall.reduceRegions(collection=VN,reducer="mean",scale=1000) 
    mdict=value.select(["VARNAME_1","mean"], ["Province","Precipitation"],retainGeometry=False).getInfo()
    nlist=[]
    for key in mdict["features"]:
        ketqua=key["properties"]
        ketqua["Year"]=year
        nlist.append(ketqua)
    return nlist    

In [ ]:
# Write a function to format the data

def outdf(start_year,end_year):
    mlist=list(map(years,range(start_year,end_year)))
    flist=[]
    for i in mlist:
        flist.extend(i)
    df=pd.DataFrame(flist)
    return df

In [ ]:
# Test the function and get data from 2000 to 2010
test=outdf(2000,2011)

In [ ]:
test.head()